### Desafio entregable: Usando APIs
<b>Autor</b>: Emanuel Benitez<br>
<b>Fecha</b>: 25-11-2023

- Buscar información en APIs públicas (i.e Twitter, NewsAPI, Spotify, Google APIS, etc).
- Extraer datos e importarlos a un dataframe realizando una exploración simple (i.e filas, columnas, tipos de datos). 

· Notebook donde se detallen los pasos seguidos

Sugerencias

- No compartir tokens personales
- Comprender el funcionamiento de APIS antes de usarla
- La limpieza de datos en apIs no es facil
- Tratar de obtener principalmente datos númericos (al menos 20 columnas y 10mil filas).

# <center>Desafio: Usando APIs </center>

<p>&nbsp;Para responder a la consigna voy a usar la Web API de Spotify, la cual contiene datos como número de seguidores, reproducciones, nombre de artista y demás. Todas las solicitudes a la API de Spotify requieren autorización, por lo que primero debo generar un token de acceso antes de poder acceder a los datos públicos.</p>

## 1. Generando token de acceso a la API

<p>&nbsp;Para autenticar y autorizar mis solicitudes a la Web API de Spotify, primero necesito generar un token de acceso usando las credenciales de mi aplicación (creada en el <a href=" https://developer.spotify.com/dashboard">tablero de desarrollo de Spotify para Developers</a>). </p>

<p> Los pasos requeridos para usar las credenciales son:</p>
    
- Enviar una solicitud POST al token endpoint de la URI
- Agregar el header "Content-Type" con el valor: "application/x-www-form-urlencoded".
- Agregar un curpo HTTP que contenga: <i>Client ID</i>, <i>Client Secret</i>, y el parametro <i>grant_type</i> configurado con el valor: "client_credentials".


<br>
<b>a. Ingreso de credenciales y generación del token</b>
<p>&nbsp;Esto va a generar el token de acceso con una duración de 60 minutos. En el cuaderno, quite mis credenciales para poder compartirlo, pero la reproductibilidad es la misma para otras credenciales de distintas web-apps generadas en la página para desarrolladores en Spotify.</p>
    

In [69]:
import requests
import json
import pandas as pd
import time

# URL del punto de extremo de token
url_del_token = "https://accounts.spotify.com/api/token"

# En estas variables coloco mis credenciales (eliminadas para seguridad)
client_id = "En esta cadena va el client ID"
client_secret = "En esta cadena va la clave Client Secret "

# Estos son los parámetros necesarios para realizar la solicitud
token_params = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

# Encabezados o headers requeridos
token_headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

token_url = "https://accounts.spotify.com/api/token"

# Realizar la solicitud POST
response = requests.post(token_url, data=token_params, headers=token_headers)

<b>b. Extrayendo el token generado</b>
<p>&nbsp;En este paso genere el token para validar las solicitudes a la API. El siguiente código deberia mostrar la respuesta en json()</p>

{'access_token': 'cadena de caracteres que corresponden al token',<br>
'token_type': 'Bearer',<br>
'experies_in': 3600}

In [135]:
#response.json()

Y en este código paso de json (leyendolo como cadena/string) a un diccionario de python, para poder acceder al token de acceso

In [71]:
diccionario_token_info = json.loads(response.text)

 Con esto ya puedo acceder al token y almacenarlo en una variable token_de_acceso:

In [72]:
token_de_acceso = diccionario_token_info['access_token']

<b>c. Ejemplo de solicitud de datos en formato JSON </b>
<p>&nbsp;Ahora puedo obtener información, cómo ejemplo, del artista usando el token de acceso generado anteriormente:</p>

In [73]:
# URL del endpoint para obtener información sobre un artista

artista_url_endpoint = "https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb"

headers = {
    "Authorization": f"Bearer {token_de_acceso}"
}

response = requests.get(artista_url_endpoint, headers=headers)
response.json()

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
 'followers': {'href': None, 'total': 8992303},
 'genres': ['alternative rock',
  'art rock',
  'melancholia',
  'oxford indie',
  'permanent wave',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
 'id': '4Z8W4fKeB5YxbusRsdQVPb',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eba03696716c9ee605006047fd',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174a03696716c9ee605006047fd',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178a03696716c9ee605006047fd',
   'width': 160}],
 'name': 'Radiohead',
 'popularity': 78,
 'type': 'artist',
 'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}

## 2. Solicitud de extracción de datos

En este paso voy a solicitar datos sobre canciones que coincidan con generos del folklore argentino.
- Zamba
- Chacarera
- Chamamé
- Cueca
- Milonga
- Gato
- Vidala
- Chaya
- Huella
- Baguala
- Payada
- Vidala Santiagueña

Para cada uno de estos generos, voy a buscar <b>n</b> número de canciones para agregar a mi DataFrame. Primero voy a extraer los datos cómo prueba, usando el url para el genero "Zamba". 

> Este <i>endpoint</i> me va a mostrar los resultados en base a la coincidencia, relevancia y popularidad de la canción en relación a lo buscado, por lo que no se debe considerar una muestra aleatoria de datos. 

Para hacer que la busqueda sea más precisa, voy a agregar el termino "<i>folklore</i>" a cada busqueda de género músical.

> El formato de busqueda va a ser: <i>api.spotify.com/v1/search?q=genre%3A<b>genero_musical</b>%2C+folklore&type=track&limit=50</i>

In [74]:
# URL del punto de extremo de búsqueda
search_url = "https://api.spotify.com/v1/search?q=genre%3A{1}%2C+folklore&type=track&limit=30".format(1,'zamba')


# Realizar la solicitud GET para buscar canciones
response = requests.get(search_url, headers=headers)

Luego de hacer la solicitud, debo entrar a los datos. Los datos de mi interes  estan "anidados" en la forma:
tracks -> items -> [n resultado]
En una linea se leeria:<br>

<code>datos_de_zamba = json.loads(respuesta.text)</code>

<code>datos_de_zamba['tracks']['items'][0]</code>
<p>A partir de esto, puedo seleccionar entonces variables de interes. El número 0 va a ser la posición del primer resultado, y si pedi 50 resultados, la ultima posición va a ser 49.</p>

Los datos que voy a tomar para mi interes son:


- <b>Duración de la canción</b>
 - datos_de_zamba['tracks']['items'][n]['duration_ms']
<br>
<br>
- <b>Nombre de la canción</b>:
 - datos_de_zamba['tracks']['items'][n]['name']
<br>
<br>
- <b>Artista</b>:
 - datos_de_zamba['tracks']['items'][n]['artists'][0]['name']
<br>
<br>
- <b>Si es explicita</b>:
 - datos_de_zamba['tracks']['items'][n]['explicit']
<br>
<br>

- <b>Popularidad</b>:
 - datos_de_zamba['tracks']['items'][n]['popularity']
<br>
<br>

- <b>Nombre del album</b>:
 - datos_de_zamba['tracks']['items'][n]['album']['name']
<br>
<br>

- <b>Tipo de album</b>:
 - datos_de_zamba['tracks']['items'][n]['album']['album_type']
<br><br>

- <b>Fecha de lanzamiento del album</b>:
 - datos_de_zamba['tracks']['items'][n]['album']['release_date']
 
 
<p>&nbsp; Ahora, lo que debo hacer es iterar n veces para extraer cada uno de estos datos, y agregarlos a cada fila de mi DataFrame junto a sus columnas correspondientes. Esto lo estaria haciendo para un solo género de los buscados, por lo que podría declarar una función que simplifique esto. 

In [75]:
zamba_data = json.loads(response.text)

def obtener_caracteristicas_canciones(datos_dict_json, genero):
    """ Toma el resultado de la búsqueda y solicitud a la Web API de Spotify,
    ya en formato diccionario (json.loads)
    y extrae las siete características de cada canción o track en la respuesta.
     También toma cómo ingreso una cadena de caracteres que agregue el género o categoría
     de este grupo de canciones.
    
    Esta función retorna una lista que puede convertirse en una tabla de Pandas.
    """
    
    canciones = []

    for track in datos_dict_json['tracks']['items']:
        cancion = {
            'nombre_cancion': track['name'],
            'artista' : track['artists'][0]['name'],
            'duracion_ms': track['duration_ms'],
            'explicita': track['explicit'],
            'popularidad': track['popularity'],
            'nombre_album': track['album']['name'],
            'tipo_album': track['album']['album_type'],
            'lanzamiento_album': track['album']['release_date'],
            'genero': str(genero)
        }
        canciones.append(cancion)

    return canciones

canciones_zamba = obtener_caracteristicas_canciones(zamba_data, "Zamba")
pd.DataFrame(canciones_zamba).head()

,nombre_cancion,artista,duracion_ms,explicita,popularidad,nombre_album,tipo_album,lanzamiento_album,genero
0,Folklore Romántico: Nada Tengo de Ti / Canción...,Campedrinos,475764,False,44,Folklore Romántico: Nada Tengo de Ti / Canción...,single,2021-07-02,Zamba
1,Cueca De La Viña Nueva - Remastered 2003,Los Chalchaleros,152786,False,17,Nuestro Folklore En Hollywood,album,2004-05-04,Zamba
2,La Parcería,Los Hermanos Toledo,133506,False,15,Folklore Los 100 Mejores Temas Vol. 1,compilation,2009-01-01,Zamba
3,Cocherito e´plaza,Los Chalchaleros,189205,False,28,Folklore argentino,album,2015-06-30,Zamba
4,Cruz del Sur,Atahualpa Yupanqui,252413,False,24,Folklore Los 100 Mejores Temas Vol. 1,compilation,2009-01-01,Zamba


<p>&nbsp;Entonces ya cree la función "obtener_caracteristicas_canciones" la cual extrae todos estos datos y los agrega en una lista. Puedo iterar en esta función para <b>extraer ejemplos de distintos generos músicales</b>.</p>

<b>b.</b> Otro paso para hacer más practico la programación, voy a crear una función que genere en enlace para cada género, usando un limite de 50 resultados (el maximo permitido por la Web API de Spotify.)


In [76]:
def generar_url(genero): 
    url = "https://api.spotify.com/v1/search?q=genre%3A{1}%2C+folklore&type=track&limit=50".format(1, genero)
    return url


generos_folklore = ["Zamba", "Chacarera","Chamame",
                    "Cueca","Milonga","Gato",
                    "Vidala","Chaya","Huella",
                    "Baguala","Payada","Vidala+Santiagueña"]
lista_urls =[]


for genero in generos_folklore:
    lista_urls.append(generar_url(genero))

lista_urls

['https://api.spotify.com/v1/search?q=genre%3AZamba%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AChacarera%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AChamame%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3ACueca%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AMilonga%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AGato%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AVidala%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AChaya%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3AHuella%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3ABaguala%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genre%3APayada%2C+folklore&type=track&limit=50',
 'https://api.spotify.com/v1/search?q=genr

<b>c.</b> Entonces ahora ya tengo la lista de las urls para solicitar generos folkloricos en la región de Argentina. Con eso ya puedo iterar en esta lista y hacer solicitudes por cada genero, y luego concatenar las listas generados por la función "<b>obtener_caracteristicas_canciones</b>" en un solo DataFrame de pandas.<br>

 El proceso para obtener los datos es:
1. Realizo una solicitud get usando el url para end-point de busqueda, con los headers que contienen el token de acceso
2. Tomo respuesta JSON como texto y lo convierto a un objeto de python usando la libreria json
3. Extraigo los datos de ese genero especifico usando la función "obtener_caracteristicas_canciones" y especifico el genero en la función
4. Agrego el diccionario generado a otro usando el metodo <b>update()</b> para unir diccionarios.
5. Creo el DataFrame con todos los datos para todos los distintos generos, usando el diccionario que es la unión de todos estos.

In [84]:
# Quiero "mapear" los valores a cada clave correspondiente en un diccionario, para la columna "genero"
generos_urls = dict(zip(generos_folklore, lista_urls))

datos_totales = []

for genero, url_endpoint in generos_urls.items():
    # Uso el endpoint para la url especificada de tal genero
    respuesta = requests.get(url_endpoint, headers=headers)
    datos_json = json.loads(respuesta.text)
    
    # Ahora extraigo los datos resultantes para este genero músical
    # y los agrego a la lista final "datos_totales" 
    datos_extraidos = obtener_caracteristicas_canciones(datos_json, genero)
    datos_totales.append(datos_extraidos)
    time.sleep(2)

Con esto ya deberia tener los datos almacenados en la variable <b>datos_totales</b>. Esta es una lista de listas que contiene cada resultado para cada genero. Si no hubo resultados para la busqueda, la lista va a estar vacia. 

## 3. Creación de tabla y exploración de los datos (EDA) básica

<p>&nbsp;Para crear el DataFrame total que contenga todos los resultados de busquedas obtenidos, debo hacer una concatenación de las listas generadas. Estas estan almacenadas en la variable "<b>datos_totales</b>" después de ejecutar el código anterior.</p>

In [128]:
dataframe_total = pd.concat([pd.DataFrame(lista_datos) for lista_datos in datos_totales], ignore_index =True)
dataframe_total.sample(5)

,nombre_cancion,artista,duracion_ms,explicita,popularidad,nombre_album,tipo_album,lanzamiento_album,genero
91,El guata de lapiz,Los Hermanos Campos,182546,False,0,Grandes Interpretes Del Folklore,compilation,1999-01-01,Cueca
84,Cuando Topea Mi Taita,Las Morenitas,158640,False,6,Estrellas del Folklore de Chile,compilation,1980,Cueca
85,La Consentida,Los Hermanos Campos,163253,False,5,Padres del Folklore (Vol. 1),album,2017-12-24,Cueca
1,Cueca De La Viña Nueva - Remastered 2003,Los Chalchaleros,152786,False,17,Nuestro Folklore En Hollywood,album,2004-05-04,Zamba
121,"Puro Chile, Pura Cueca",Grupo Alerzal de Los Ríos,143746,False,0,Estudio y Música de Nuestro Folklore (Vol 2),compilation,2018-02-01,Cueca


>Con esto ya estarian cumplidas las consignas pedidas. Ahora, para una exploración de datos inicial y básica, voy a dar una vista general del dataframe con los atributos info() y describe(). 

In [131]:
dataframe_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   nombre_cancion     130 non-null    object
 1   artista            130 non-null    object
 2   duracion_ms        130 non-null    int64 
 3   explicita          130 non-null    bool  
 4   popularidad        130 non-null    int64 
 5   nombre_album       130 non-null    object
 6   tipo_album         130 non-null    object
 7   lanzamiento_album  130 non-null    object
 8   genero             130 non-null    object
dtypes: bool(1), int64(2), object(6)
memory usage: 8.4+ KB


In [132]:
dataframe_total.describe()

,duracion_ms,popularidad
count,130.000000,130.000000
mean,164050.815385,6.638462
std,49772.149190,7.942627
min,87160.000000,0.000000
25%,128206.250000,1.000000
50%,161138.500000,4.000000
75%,185839.500000,9.000000
max,475764.000000,44.000000


In [133]:
dataframe_total['genero'].value

Zamba      50
Cueca      50
Chamame    30
Name: genero, dtype: int64

In [134]:
dataframe_total

,nombre_cancion,artista,duracion_ms,explicita,popularidad,nombre_album,tipo_album,lanzamiento_album,genero
0,Folklore Romántico: Nada Tengo de Ti / Canción...,Campedrinos,475764,False,44,Folklore Romántico: Nada Tengo de Ti / Canción...,single,2021-07-02,Zamba
1,Cueca De La Viña Nueva - Remastered 2003,Los Chalchaleros,152786,False,17,Nuestro Folklore En Hollywood,album,2004-05-04,Zamba
2,La Parcería,Los Hermanos Toledo,133506,False,15,Folklore Los 100 Mejores Temas Vol. 1,compilation,2009-01-01,Zamba
3,Cocherito e´plaza,Los Chalchaleros,189205,False,28,Folklore argentino,album,2015-06-30,Zamba
4,Cruz del Sur,Atahualpa Yupanqui,252413,False,24,Folklore Los 100 Mejores Temas Vol. 1,compilation,2009-01-01,Zamba
...,...,...,...,...,...,...,...,...,...
125,El Guata E' Vino,Los Hermanos Campos,157173,False,1,Padres del Folklore (Vol. 1),album,2017-12-24,Cueca
126,El Morro de Arica,Los Hermanos Campos,87160,False,24,Las 100 Mejores del Folklore Chileno,compilation,2018-01-01,Cueca
127,No Hay Como el Roto Chileno - En Directo,Daniel Muñoz,116160,False,11,Estudio y Música de Nuestro Folklore (Vol. 1),compilation,2018-02-01,Cueca
128,El Celoso,Los Hermanos Campos,107186,False,23,Las 100 Mejores del Folklore Chileno,compilation,2018-01-01,Cueca


<p>&nbsp;Podemos ver que las muestras son para los generos Zamba, Cueca y Chamame. No se obtuvieron más datos resultantes de la busqueda, por lo que habria que ver abarcando más datos. También vemos que en los datos obtenidos la duración promedio de una canción es de 164050 milisegundos. Esto es 164 o 2 minutos y 44 segundos, lo que indica que no hay valores atipicos evidentes en la duración. </p> 

In [137]:
dataframe_total.to_csv('sptfy_datos_solicitados.csv')